# Allison CNN Sentence Classification
## Applied Deep Learning II
## Biweekly Report 1 - Jan 26 2022
In this notebook, I implement the model presented in the paper titled 'Convolutional Neural Networks for Sentence Classification' (Yoon Kim 2014) and then evaluate it on a movie review dataset from rotten tomatoes found here: https://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz. I wanted to understand this model because it has become a baseline for new text classification architectures due to good classification performance across text classification tasks. 


I follow the tutorial presented on https://opendatascience.com/implementing-a-cnn-for-text-classification-in-tensorflow/ which implements Yoon Kim's original model (done in python 2.7) using tensorflow. TensorFlow was my choice of library, because while I have experimented with PyTorch, I have never touched tensorflow and I want to gain a better baseline understanding of it this semester.

In [1]:
import tensorflow as tf
import numpy as np
import os
import datetime
from pprint import pprint
import time

In [2]:
print(tf.__version__)
print(np.__version__)

1.14.0
1.16.4


## EDA on Movie Review Dataset

Download the dataset from cs.cornell website.

In [3]:
url = 'https://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz'
!wget -P './data' $url
!tar xvzf 'data/rt-polaritydata.tar.gz' -C 'data/'

dataset_dir = os.path.join('./data/rt-polaritydata')
!rm -r ./data/rt-polaritydata.tar.gz

--2022-01-26 01:49:25--  https://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 487770 (476K) [application/x-gzip]
Saving to: ‘./data/rt-polaritydata.tar.gz’

rt-polaritydata.tar 100%[===================>] 476.34K  1.49MB/s    in 0.3s    

2022-01-26 01:49:25 (1.49 MB/s) - ‘./data/rt-polaritydata.tar.gz’ saved [487770/487770]

x rt-polaritydata.README.1.0.txt
x rt-polaritydata/rt-polarity.neg
x rt-polaritydata/rt-polarity.pos


In [4]:
os.listdir(dataset_dir)

['rt-polarity.neg', 'rt-polarity.pos']

Load data and print some examples. We see that there are 5331 positive examples in this dataset and 5331 negative examples.

In [5]:
pos_examples = list(open(os.path.join(dataset_dir, 'rt-polarity.pos'), encoding='unicode_escape').readlines())
pos_examples = [review.strip(' \n ') for review in pos_examples]
pprint(pos_examples[:5])
print("pos statements: ", len(pos_examples))
print("----------------------------------")
neg_examples = list(open(os.path.join(dataset_dir, 'rt-polarity.neg').lower().strip(), encoding = 'unicode_escape').readlines())
neg_examples = [review.strip(' \n ') for review in neg_examples]
pprint(neg_examples[:5])
print("neg statements: ", len(neg_examples))


['the rock is destined to be the 21st century\'s new " conan " and that he\'s '
 'going to make a splash even greater than arnold schwarzenegger , jean-claud '
 'van damme or steven segal .',
 'the gorgeously elaborate continuation of " the lord of the rings " trilogy '
 'is so huge that a column of words cannot adequately describe '
 "co-writer/director peter jackson's expanded vision of j . r . r . tolkien's "
 'middle-earth .',
 'effective but too-tepid biopic',
 'if you sometimes like to go to the movies to have fun , wasabi is a good '
 'place to start .',
 "emerges as something rare , an issue movie that's so honest and keenly "
 "observed that it doesn't feel like one ."]
pos statements:  5331
----------------------------------
['simplistic , silly and tedious .',
 "it's so laddish and juvenile , only teenage boys could possibly find it "
 'funny .',
 'exploitative and largely devoid of the depth or sophistication that would '
 'make watching such a graphic treatment of the crim

## Label Data

In [6]:
def label_data(pos_examples, neg_examples):
    x_text = pos_examples+neg_examples
    y = np.array([[0, 1]]*len(pos_examples)+[[1, 0]]*len(neg_examples))
    return[x_text, y]

x_text, y = label_data(pos_examples, neg_examples)

## Build Vocabulary
First, tokenize the words using Tokenizer, which vectorizes a text corpus by turning each text into a sequence of integers (with each integer being the index of a token in a dictionary). Punctuation is removed by default. Then, create a vocabulary dictionary and see that there are 19,498 vocabulary words in the corpus. The longest review has a length of 59 words, so all the word vectors have a length of 59.

In [7]:
max_doc_len = max([len(x.split(" ")) for x in x_text])
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(x_text)
X = tokenizer.texts_to_sequences(x_text)
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=max_doc_len, padding='post', truncating='post')

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size = 0.15, random_state=8)
VOCAB_SIZE = len(tokenizer.word_index)

print(f'Vector Lengths: {max_doc_len}')
print(f'Vocabulary Size: {VOCAB_SIZE}')
print(f"Train/Dev split: {len(y_train)}/{len(y_dev)}")

# Write out vocabulary dictionary
vocabulary = tokenizer.index_word
vocabulary.values()
with open('./data/vocab.txt', 'w') as f:
    f.write(str(vocabulary))

Vector Lengths: 59
Vocabulary Size: 19498
Train/Dev split: 9062/1600


## Text CNN Model
* First layers: embed words into low-dimensional vectors
* Next layers: convolve over the embedded word vectors using filter sizes of 3, 4, and 5
* Max-pool the result of the convolutional later into a long feature vector
* Add dropout regularization
* Classify the results using a softmax 

Note that pre-trained word2vec vectors are not used for word embeddings and are learned from scratch.

Put into a TextCNN class to allow for various hyperparameter configurations.

Dropout is a method to regularize CNNs. The dropout layer stochastically "disables" a fraction of neurons, which forces them to learn individually useful features. "dropout_keep_prob" is defined as the fraction of neurons we keep enabled. During evaluation dropout is disabled.

In [9]:

class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.compat.v1.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.compat.v1.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.compat.v1.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random.uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                # W is the embedding matrix learned during training. Initialize as random unif dist
                W = tf.Variable(tf.random.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool2d(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.compat.v1.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [10]:
# Batch iteration function to shuffle training batches. I think there is a tensorflow module for this, 
# but need to look into it further.

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

## Train
Iterate over batches of data, call train_step for each batch, occasionally evaluate and checkpoint model.

In [11]:
## Parameters
# Data loading params
DEV_SAMPLE_PERCENTAGE = 0.1
POSITIVE_DATA_FILE = "./data/rt-polaritydata/rt-polarity.pos"
NEGATIVE_DATA_FILE = "./data/rt-polaritydata/rt-polarity.neg"

# Model Hyperparameters
EMBEDDING_DIM = 128
FILTER_SIZES = "3, 4, 5"
NUM_FILTERS = 128
DROPOUT_KEEP_PROB = 0.5
L2_REG_LAMBDA = 0.0

# Training parameters
BATCH_SIZE = 64
NUM_EPOCHS = 200
EVALUATE_EVERY = 100
CHECKPOINT_EVERY = 100
NUM_CHECKPOINTS =- 5
ALLOW_SOFT_PLACEMENT = True
LOG_DEVICE_PLACEMENT = False

In [12]:
def train(x_train, y_train, x_dev, y_dev):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.compat.v1.ConfigProto(
            allow_soft_placement=ALLOW_SOFT_PLACEMENT,
            log_device_placement=LOG_DEVICE_PLACEMENT
        )
        sess = tf.compat.v1.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1],
                vocab_size=VOCAB_SIZE+1,
                embedding_size=EMBEDDING_DIM,
                filter_sizes=list(map(int, FILTER_SIZES.split(","))),
                num_filters=NUM_FILTERS,
                l2_reg_lambda=L2_REG_LAMBDA)

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.compat.v1.train.AdamOptimizer(1e-3)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Keep track of gradient values and sparsity (optional)
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.compat.v1.summary.histogram("{}/grad/hist".format(v.name.replace(":", "_")), g)
                    sparsity_summary = tf.compat.v1.summary.scalar("{}/grad/sparsity".format(v.name.replace(":", "_")), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.compat.v1.summary.merge(grad_summaries)

            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            print(f"run: {timestamp}")
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
            print("Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.compat.v1.summary.scalar("loss", cnn.loss)
            acc_summary = tf.compat.v1.summary.scalar("accuracy", cnn.accuracy)

            # Train Summaries
            train_summary_op = tf.compat.v1.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.compat.v1.summary.FileWriter(train_summary_dir, sess.graph)

            # Dev summaries
            dev_summary_op = tf.compat.v1.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.compat.v1.summary.FileWriter(dev_summary_dir, sess.graph)

            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.compat.v1.train.Saver(tf.compat.v1.global_variables(), max_to_keep=NUM_CHECKPOINTS)

            # Initialize all variables
            sess.run(tf.global_variables_initializer())

            def train_step(x_batch, y_batch):
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: DROPOUT_KEEP_PROB
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                train_summary_writer.add_summary(summaries, step)

            def dev_step(x_batch, y_batch, writer=None):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                if writer:
                    writer.add_summary(summaries, step)

            # Generate batches
            batches = batch_iter(
                list(zip(x_train, y_train)), BATCH_SIZE, NUM_EPOCHS)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch)
                current_step = tf.compat.v1.train.global_step(sess, global_step)
                if current_step % EVALUATE_EVERY == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev, writer=dev_summary_writer)
                    print("")
                if current_step % CHECKPOINT_EVERY == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))

In [13]:
def main(argv=None):
    train(X_train, y_train, X_dev, y_dev)

if __name__ == "__main__":
    tf.compat.v1.app.run()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2022-01-26 01:49:28.827094: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-26 01:49:28.827358: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.
W0126 01:49:28.907726 4454043136 deprecation.py:506] From /var/folders/33/49tykqg9645_cd5gw3v25dfh0000gn/T/ipykernel_21279/2205772446.py:59: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0126 01:49:28.929696 4454043136 deprecation_wrapper.py:119] From /var/folders/33/49tykqg9645_cd5gw3v25dfh0000gn/T/ipykernel_21279/2205772446.py:63: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0126 01:49:30.438992 4454043136 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



...

I0126 01:55:43.806762 4454043136 checkpoint_management.py:95] /Users/allisonliu/Documents/Spring2022/APPM5720/biweekly-report-1-reign/runs/1643186971/checkpoints/model-1900 is not in all_model_checkpoint_paths. Manually adding it.


Saved model checkpoint to /Users/allisonliu/Documents/Spring2022/APPM5720/biweekly-report-1-reign/runs/1643186971/checkpoints/model-1900

2022-01-26T01:55:44.049465: step 1901, loss 0.0460076, acc 1
2022-01-26T01:55:44.228874: step 1902, loss 0.0463365, acc 0.984375
2022-01-26T01:55:44.424710: step 1903, loss 0.0821136, acc 0.96875
2022-01-26T01:55:44.798651: step 1904, loss 0.0995566, acc 0.96875
2022-01-26T01:55:45.087051: step 1905, loss 0.0858001, acc 0.96875
2022-01-26T01:55:45.327187: step 1906, loss 0.0349453, acc 1
2022-01-26T01:55:45.533576: step 1907, loss 0.0568097, acc 0.96875
2022-01-26T01:55:45.745369: step 1908, loss 0.0213526, acc 1
2022-01-26T01:55:45.981983: step 1909, loss 0.0461354, acc 0.984375
2022-01-26T01:55:46.170403: step 1910, loss 0.165708, acc 0.9375
2022-01-26T01:55:46.351895: step 1911, loss 0.0520626, acc 0.984375
2022-01-26T01:55:46.567997: step 1912, loss 0.0534356, acc 1
2022-01-26T01:55:46.756959: step 1913, loss 0.0482097, acc 1
2022-01-26T01:55:46.

I0126 01:56:03.315430 4454043136 checkpoint_management.py:95] /Users/allisonliu/Documents/Spring2022/APPM5720/biweekly-report-1-reign/runs/1643186971/checkpoints/model-2000 is not in all_model_checkpoint_paths. Manually adding it.


Saved model checkpoint to /Users/allisonliu/Documents/Spring2022/APPM5720/biweekly-report-1-reign/runs/1643186971/checkpoints/model-2000

2022-01-26T01:56:03.578851: step 2001, loss 0.134933, acc 0.9375
2022-01-26T01:56:03.775584: step 2002, loss 0.044592, acc 1
2022-01-26T01:56:03.958809: step 2003, loss 0.0716052, acc 0.96875
2022-01-26T01:56:04.233597: step 2004, loss 0.098773, acc 0.96875
2022-01-26T01:56:04.483656: step 2005, loss 0.10013, acc 0.953125
2022-01-26T01:56:04.763121: step 2006, loss 0.0336026, acc 0.984375
2022-01-26T01:56:05.005384: step 2007, loss 0.0255258, acc 1
2022-01-26T01:56:05.242869: step 2008, loss 0.0703787, acc 0.96875
2022-01-26T01:56:05.442436: step 2009, loss 0.059538, acc 0.96875
2022-01-26T01:56:05.719969: step 2010, loss 0.0467266, acc 0.984375
2022-01-26T01:56:05.903212: step 2011, loss 0.0560277, acc 0.984375
2022-01-26T01:56:06.132286: step 2012, loss 0.064519, acc 0.96875
2022-01-26T01:56:06.358670: step 2013, loss 0.065719, acc 0.984375
2022-01

KeyboardInterrupt: 

## Evaluation
I learned how to use TensorBoard to view my results. It's a very, cool and useful module that I have never gotten to use before, but outputs a lot of useful metrics in an interactive format hosted on a local server. Version control was horrendous to use this module, but I finally figured out it was because I had two versions of TensorBoard downloaded on my computer.
<img src = "./runs/tensorboard_ex.png" width=70%>

#### Test Accuracy
I was able to achieve about 75% accuracy after training for approximately 4.2k timesteps. The accuracy begins decreasing after this point due to the model overfitting. I was pretty pleased with these results, though they were lower than the results presented in the Yoon Kim paper of 81%, because I had only trained for about 6 mins. Using pretrained word2vec vectors might improve my model.

<img src = "./runs/test_data_acc.png" width=70%>

#### Training Accuracy
I was able to achieve a training accuracy of 100% after 2.56k timesteps.

<img src = "./runs/train_data_acc.png" width=70%>

In [16]:
!tensorboard --logdir ./runs/1643162486/summaries/dev

TensorBoard 1.14.0 at http://Allisons-Macbook.local:6006/ (Press CTRL+C to quit)
^C


In [17]:
!tensorboard --logdir ./runs/1643162486/summaries/train

TensorBoard 1.14.0 at http://Allisons-Macbook.local:6006/ (Press CTRL+C to quit)
^C


### Final Remarks
I would love to test the performance of this model on other datasets and compare them to those achieved in the Kim Yoon paper. I look forward to seeing the results of my groupmates and I's combined python notebook where we integrate the three models we each explored more in-depth.

### References
* Yoon Kim paper: https://arxiv.org/pdf/1408.5882.pdf
* Tutorial: https://opendatascience.com/implementing-a-cnn-for-text-classification-in-tensorflow/
* Colab Text Classification: https://www.tensorflow.org/tutorials/keras/text_classification
* Understanding CNNs for NLP: https://www.kdnuggets.com/2015/11/understanding-convolutional-neural-networks-nlp.html 
* Sensitivity Analysis of CNNs for Sentence Classification (Zhang et al. 2015): https://arxiv.org/abs/1510.03820